In [213]:
import os
import pandas as pd
from pathlib import Path

#weather
weather_df = pd.read_csv("../data/raw/meteostat/dia_2023-2024_hourly.csv")

#departures
bts_departures = Path("../data/raw/bts/departure_stats")
departure_files = list(bts_departures.glob("*csv"))
departures_df = pd.concat((pd.read_csv(f) for f in departure_files))

#bts t100
t100_column_key = ["Year","Month", "OA_Alpha_Code", "OA_Unique_Numeric_Code",
                   "OA_World_Area_Code", "OA_City_Name", "DA_Alpha_Code","DA_Airport_Numeric_Code",
                   "DA_World_Area_Code", "DA_City_Name", "Carrier_Alpha_Code", "Carrier_Entity_Code", 
                   "OAI_Group_Code", "Distance","Service_Class", "Aircraft_Group","Aircraft_Type",
                   "Aircraft_Configuration","Departures_Performed", "Departures_Scheduled", "Available_Capacity",
                   "Available_Seats", "Passenger_Transported","Freight_Transported","Mail_Transported","Ramp-Ramp_Minutes",
                   "Airborne_Minutes", "Carrier_World_Area_Code"]
bts_t100 = Path("../data/raw/bts/t100")
t100_files = list(bts_t100.glob("*.asc"))
t100_df = pd.concat((pd.read_table(f, sep= "|", header = None) for f in t100_files))
t100_df = t100_df.drop(columns=[28])
t100_df.columns = t100_column_key
#filter down to just DIA rows (DEN but ill never consider it such)
destination = t100_df.loc[t100_df['DA_City_Name'] == "Denver, CO"]
arrival = t100_df.loc[t100_df['OA_City_Name'] == "Denver, CO"]
t100_df = pd.concat([destination, arrival], ignore_index=True)

# print(weather_df.head(2))
# print("departures")
# print(departures_df.head(2))
# print("t100")
# print(t100_df.head(2))

In [214]:
#make date column into datetime and sort depart chronologically
# print(departures_df.info())

departures_df.rename(columns={'Date (MM/DD/YYYY)': 'Date'}, inplace= True)
departures_df['Datetime'] = pd.to_datetime(departures_df['Date'] + ' ' + departures_df['Scheduled departure time'])
departures_df.sort_values(by='Datetime', ascending=True, inplace=True)
departures_df.drop(columns = ['Tail Number'],inplace=True)
departures_df.dropna(how='any',inplace=True)
# print(departures_df.head(3))

#datetime for weather stats
# print(weather_df.info())
weather_df['Datetime'] = pd.to_datetime(weather_df['time'])
weather_df.sort_values(by='Datetime',ascending=True, inplace=True)
weather_df.drop(columns=['time','tsun','coco','wpgt','snow'], inplace=True)
weather_df.fillna(0, inplace=True)
# print(weather_df.head(3))

#datetime for t100 turn monthly 
# print(t100_df.shape)
# print(t100_df.info())
# print(t100_df.head(2))
t100_arr = t100_df[t100_df['DA_Alpha_Code'] == "DEN"] 
t100_dep = t100_df[t100_df['OA_Alpha_Code'] == 'DEN']

t100_monthly_arr = (t100_arr.groupby(['Year','Month'], as_index=False).agg(
    monthly_passenger=('Passenger_Transported','sum'),
    monthly_freight=('Freight_Transported','sum'),
    monthly_dep = ('Departures_Performed','sum'),
    monthly_seats = ('Available_Seats','sum')
))

t100_monthly_dep = (t100_dep.groupby(['Year','Month'], as_index=False).agg(
    monthly_passenger=('Passenger_Transported','sum'),
    monthly_freight=('Freight_Transported','sum'),
    monthly_dep = ('Departures_Performed','sum'),
    monthly_seats = ('Available_Seats','sum')
))

t100_monthly= pd.merge(
    t100_monthly_arr.add_suffix('_arr'),
    t100_monthly_dep.add_suffix('_dep'),
    left_on=['Year_arr', 'Month_arr'],
    right_on=['Year_dep', 'Month_dep'],
    how='outer'
    )

t100_monthly.rename(columns={
    'Year_arr': 'Year',
    'Month_arr': 'Month',
    'monthly_dep_dep': 'total_dep',
    'monthly_dep_arr': 'total_arr'
}, inplace=True)


t100_monthly['load_factor_arr']=(
    t100_monthly['monthly_passenger_arr']/t100_monthly['monthly_seats_arr']
)

t100_monthly['load_factor_dep']=(
    t100_monthly['monthly_passenger_dep']/t100_monthly['monthly_seats_dep']
)

t100_monthly.drop(columns=['Year_dep','Month_dep'], inplace=True)
t100_monthly['Datetime'] = pd.to_datetime(t100_monthly['Year'].astype(str) +'-'+t100_monthly['Month'].astype(str))
t100_monthly.sort_values(by='Datetime', ascending=True, inplace=True)
print(t100_monthly.head(2))

   Year  Month  monthly_passenger_arr  monthly_freight_arr  total_arr  \
0  2023      1                2552684             25223364      23321   
1  2023      2                2479564             23885477      21750   

   monthly_seats_arr  monthly_passenger_dep  monthly_freight_dep  total_dep  \
0            3317339                2566847             20670400      23320   
1            3086486                2461473             19520583      21757   

   monthly_seats_dep  load_factor_arr  load_factor_dep   Datetime  
0            3319085         0.769497         0.773360 2023-01-01  
1            3089684         0.803361         0.796675 2023-02-01  


In [215]:
#Merge all 3 dfs to one
# print(departures_df.isna().sum())
# print(weather_df.isna().sum())
# print(t100_monthly.isna().sum())

departures_df = pd.merge_asof(
    departures_df,
    weather_df,
    on='Datetime',
    direction='nearest',
)

# departures_df = pd.merge_asof(
#     departures_df,
#     t100_monthly,
#     on ='Datetime',
#     direction='backward'
# )

departures_df['ym'] = departures_df['Datetime'].dt.to_period('M')
t100_monthly['ym'] = t100_monthly['Datetime'].dt.to_period('M')

departures_df = departures_df.merge(
    t100_monthly.drop(columns=['Datetime']),
    on='ym',
    how='left',
)

departures_df.drop(columns=['ym','Date','Year','Month'], inplace=True)
departures_df = departures_df[['Datetime'] + [col for col in departures_df.columns if col != 'Datetime']]
# print(departures_df.head(3))
# print(departures_df.tail(3))
print(departures_df.info())

processed_file = Path("../data/processed")
departures_df.to_csv(processed_file /"flight_log.csv",index=False)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436725 entries, 0 to 436724
Data columns (total 33 columns):
 #   Column                                    Non-Null Count   Dtype         
---  ------                                    --------------   -----         
 0   Datetime                                  436725 non-null  datetime64[ns]
 1   Carrier Code                              436725 non-null  object        
 2   Flight Number                             436725 non-null  float64       
 3   Destination Airport                       436725 non-null  object        
 4   Scheduled departure time                  436725 non-null  object        
 5   Actual departure time                     436725 non-null  object        
 6   Scheduled elapsed time (Minutes)          436725 non-null  float64       
 7   Actual elapsed time (Minutes)             436725 non-null  float64       
 8   Departure delay (Minutes)                 436725 non-null  float64       
 9   Wheels-off time